In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from my_utils import Dictionary

n_unique = 10

src_dict = Dictionary(['<EOS>'])
tgt_dict = Dictionary(['<BOS>', '<EOS>'])
for n in range(n_unique):
    src_dict.add_word(str(n))
    tgt_dict.add_word(str(n))

In [3]:
from my_utils.toy_data import invert_seq
train = invert_seq(5000, n_unique=n_unique)
test = invert_seq(100, n_unique=n_unique)

In [4]:
import torch
from my_utils import DataLoader
from torch_models.utils import seq2seq

def numericalize(dataset, src_dict, tgt_dict):
    numericalized = [([src_dict(s) for s in src], [tgt_dict(t) for t in tgt]) for src, tgt in dataset]
    return numericalized

# device = 'cuda:0'
device = torch.device('cpu')
trans_func = seq2seq(device)

train_loader = DataLoader(numericalize(train, src_dict, tgt_dict), batch_size=64, trans_func=trans_func)
test_loader = DataLoader(numericalize(test, src_dict, tgt_dict), batch_size=50, trans_func=trans_func)

In [17]:
from torch_models import AttnSeq2Seq

embed_size=64
dropout = 0
model = AttnSeq2Seq(embed_size=embed_size, hidden_size=embed_size, src_vocab_size=len(src_dict), tgt_vocab_size=len(tgt_dict),
                    src_EOS=src_dict('<EOS>'), tgt_BOS=tgt_dict('<BOS>'), tgt_EOS=tgt_dict('<EOS>'),
                    num_layers=2, bidirectional=False, dropout=dropout, rnn='lstm')
print(model)

AttnSeq2Seq(
  (encoder): RNNEncoder(
    (embedding): Embedding(12, 64, padding_idx=11)
    (rnn): LSTM(64, 64, num_layers=2)
  )
  (decoder): RNNEncoder(
    (embedding): Embedding(13, 64, padding_idx=12)
    (rnn): LSTM(64, 64, num_layers=2)
  )
  (generator): MLP(
    (fc_out): Linear(in_features=64, out_features=12, bias=True)
    (dropout): Dropout(p=0)
    (criterion): CrossEntropyLoss()
    (activation): Tanh()
  )
  (attn_hidden): Linear(in_features=128, out_features=64, bias=True)
  (attention): DotAttn()
)


In [18]:
%%time
from my_utils import Trainer, EvaluatorSeq
from torch.optim import Adam, SGD

optimizer = Adam(model.parameters())
evaluator = EvaluatorSeq(model, test_loader, measure='BLEU')

trainer = Trainer(model, train_loader)
trainer.train(optimizer, max_epoch=5,
              evaluator=evaluator, score_monitor=None, show_log=True)

/Users/linghan/anaconda/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


epoch 0  	loss: 2.0030362696587285	BLEU: 0.16817838069643742	
epoch 1  	loss: 1.2276613078539884	BLEU: 0.39275780857520876	
epoch 2  	loss: 0.5726368857335441	BLEU: 0.7509030034714899	
epoch 3  	loss: 0.23478301680540736	BLEU: 0.8509085733994385	
epoch 4  	loss: 0.11651142912952206	BLEU: 0.9155565401072485	
CPU times: user 49.5 s, sys: 766 ms, total: 50.2 s
Wall time: 18.7 s


In [19]:
assert 0.9 < evaluator.evaluate()

In [20]:
iter(train_loader)
l = 10
inputs, targets = next(train_loader)
inputs = inputs[:l]
targets = targets[:l]
generated = model.predict(inputs)
print('======= input ======')
for seq in inputs:
    print([src_dict[s.item()] for s in seq])
print('======= output ======')
for seq in generated[:l]:
    print([tgt_dict[s] for s in seq])

======= input ======
['4', '2', '3']
['4', '2', '5', '9']
['0', '2', '7', '1']
['5', '5', '9', '1', '6']
['5', '1', '3', '8', '0']
['0', '9', '4', '2']
['8', '8', '5', '7', '9']
['0', '5', '0', '0']
['7', '3', '8', '9']
['8', '4', '4', '6']
======= output ======
['3', '2', '4']
['9', '5', '2', '4']
['1', '7', '2', '0']
['6', '1', '9', '5', '5']
['0', '8', '3', '1', '5']
['2', '4', '9', '0']
['9', '7', '5', '8', '8']
['0', '0', '5', '0']
['9', '8', '3', '7']
['6', '4', '4', '8']
